# Robomind Academy

Robomind Academy was created to teach (young) persons Computational Thinking. But hand-in-hand with
with this primary goal we as teachers learn how these persons learn Computational Thinking. This knowledge allows us to to improve our methods of teaching and the circle is closed.

Robomind Academy collects data about the its users and stores this for a short period of time. This data is used in the normal operation of the teaching process but an anonymized version of this data is stored for later analysis and research.The anonymized data is stored for a longer period of time and includes things like the number of runs per person per Challenge and when they were run, the Solution in each run used and the performance of the final Solutions. 

## The Data

We will use the data from about 5000 students using the Basis1 course in Robomind Academy. This is an introductory course in Computational Thinking. The following reads the data and displays the first five rows.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dataset = pd.read_csv("basis1.gz")

dataset.head()

,storyline,person,cumtime,count
0,Basis_1/Getting started/1,IPupil000126,74.0,1
1,Basis_1/Getting started/1,IPupil000127,183.0,1
2,Basis_1/Getting started/1,IPupil000128,124.0,1
3,Basis_1/Getting started/1,IPupil000129,147.0,3
4,Basis_1/Getting started/1,IPupil000130,56.0,3


For these demonstration notebooks, the Robomind Academy data was simplified to just describe the cumulative time a person works on a storyline item and the number of time he or she runs a program before solving the Challenge in the storyline item.

As the first five line show, the simplified data contains the name/id of the storyline, the anonymized name/id the person, the total time (cumtime) the person took before coming up with the solution and the number of runs (count) it took.

## Grading Pupils

In this notebook we try to develop a grading system for pupils and using those grades try to determine what are good and bad Storyline-items/Challenges. The two indicators used are 'cumtime' and 'count'. 

As a first step we calculate the averages of 'cumtime' and 'count' for each storyline-item.



In [2]:
slis = dataset.groupby(['storyline']).describe()
slis.head()

count                                                    \
                      count       mean        std  min  25%  50%   75%    max   
storyline                                                                       
Basis_1/Factories/1  5432.0   7.659978   8.298906  1.0  2.0  5.0  10.0   85.0   
Basis_1/Factories/2  5374.0   1.551544   3.034427  1.0  1.0  1.0   1.0   68.0   
Basis_1/Factories/3  5288.0   4.873109   5.956526  1.0  1.0  3.0   6.0   74.0   
Basis_1/Factories/4  5211.0   2.962771   6.139122  1.0  1.0  1.0   2.0   87.0   
Basis_1/Factories/5  4079.0  10.517038  12.317869  1.0  3.0  7.0  13.0  158.0   

                    cumtime                                                    \
                      count        mean         std  min    25%    50%    75%   
storyline                                                                       
Basis_1/Factories/1  5432.0  475.006996  505.635286  0.0  193.0  315.0  553.0   
Basis_1/Factories/2  5374.0   62.799963  165.458768  1.0   17.0   28.0   48.0   
Basis_1/Factories/3  5288.0  201.389372  257.189134  0.0   78.0  128.0  217.0   
Basis_1/Factories/4  5211.0  115.531568  255.370727  0.0   24.0   39.0   73.0   
Basis_1/Factories/5  4079.0  430.912724  557.539334  0.0  140.0  257.0  510.0   

                              
                         max  
storyline                     
Basis_1/Factories/1   7438.0  
Basis_1/Factories/2   4255.0  
Basis_1/Factories/3   4473.0  
Basis_1/Factories/4   4491.0  
Basis_1/Factories/5  10877.0

Here we are grading persons based on 'cumtime', or the cummulated time that person has spend on a storyline item before executing a correct Solution. Was score a person/storyline item with 1 point if the person took longer then the 75% percentile of 'cumtime' (not so good) and 3 points if the person took less then the 25% percentile (fast). In all other cases a score of 2 is assigned.The following code adds and extra 'score' column to our dataset.

In [3]:
# create two python dictionaries to quickly determine scoring lines
twentyfives = {}
seventyfives = {}
for index, row in slis.iterrows():
    twentyfive = row['cumtime']['25%']
    seventyfive = row['cumtime']['75%']
    twentyfives[index] = twentyfive
    seventyfives[index] = seventyfive
# create a new score series
scores = []
for index, row in dataset.iterrows():
    cumtime = row['cumtime']
    if cumtime < twentyfives[row['storyline']]:
        score = 3
    elif cumtime > seventyfives[row['storyline']]:
        score = 1
    else:
        score = 2
    scores.append(score)
# add 'score' column to dataset
dataset['score'] = pd.Series(scores)
dataset.head()

,storyline,person,cumtime,count,score
0,Basis_1/Getting started/1,IPupil000126,74.0,1,3
1,Basis_1/Getting started/1,IPupil000127,183.0,1,2
2,Basis_1/Getting started/1,IPupil000128,124.0,1,2
3,Basis_1/Getting started/1,IPupil000129,147.0,3,2
4,Basis_1/Getting started/1,IPupil000130,56.0,3,3


Now we have scored every storyline item for every person, we can easily calculate an overall score for each person that has completed course Basic1.

In [5]:
scoresbasis1 = dataset[['person','score']].groupby(['person']).describe()
scoresbasis1.head()

score                                               
             count      mean       std  min   25%  50%   75%  max
person                                                           
APupil265619   1.0  2.000000       NaN  2.0  2.00  2.0  2.00  2.0
APupil265965   7.0  2.285714  0.755929  1.0  2.00  2.0  3.00  3.0
APupil266127  11.0  2.454545  0.522233  2.0  2.00  2.0  3.00  3.0
APupil266153   1.0  2.000000       NaN  2.0  2.00  2.0  2.00  2.0
APupil266229   2.0  2.500000  0.707107  2.0  2.25  2.5  2.75  3.0